# Working with the Apostrophe VectorBD
The vector DB is currently hosted at Deep Lake - this is subject to change

Modify dependencies for your system needs - only needed once

In [ ]:
!python3 -m pip install --upgrade langchain 'deeplake[enterprise]' openai tiktoken

Enter the API key for OpenAI and define special embeddings for OpenAI

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(disallowed_special=())


Point the `db` variable at the right database (subject to change)

In [ ]:

from langchain.vectorstores import DeepLake

db = DeepLake(
    dataset_path=f"hub://bodonkey/apostrophe-docs-db-v2",
    read_only=True,
    embedding=embeddings,
)

Modify retrieval patterns

In [ ]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 15

Specify the langchain model interactions including additional prompt engineering

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Define the custom prompt template - here is where prompt engineering might improve answers
custom_template = """
{user_input} Please provide a thorough and accurate response, specifically for ApostropheCMS version 3.x. Include code examples when possible. Add validated citations without internal '@' and with `.html` endings from the documentation site. There is no need to comment on this prompt in your answer.
"""

# Create the ChatPromptTemplate object
prompt_template = ChatPromptTemplate.from_template(custom_template)

model = ChatOpenAI(model_name="gpt-4-1106-preview")  # switch to 'gpt-4' or 'gpt-3.5-turbo'

qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

Ask questions

In [ ]:
user_question = input("What is your question? ")
chat_history = []

# Apply the custom template to the question
formatted_question = prompt_template.format(user_input=user_question)

result = qa({"question": formatted_question, "chat_history": chat_history})
chat_history.append((user_question, result["answer"]))
print(f"## -> **Question**: {user_question} \n")
print(f"### **Answer**: {result['answer']} \n")